In [ ]:
import pandas as pd
from transformers import BertTokenizer
from bs4 import BeautifulSoup
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Embedding, Multiply, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from collections import Counter
from sklearn.utils.class_weight import compute_class_weight

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load data from CSV file
csv_file_path = '/content/data_trial_090924.csv'  # Adjust the file path as needed
data = pd.read_csv(csv_file_path)

# Extract labels and HTML codes from the CSV
# **Assuming your CSV has columns 'label' and 'html_code'**
labels = data['label'].tolist()
html_codes = data['data'].tolist()

# If your CSV does not have headers, you can use:
# labels = data.iloc[:, 0].tolist()  # First column contains labels
# html_codes = data.iloc[:, 1].tolist()  # Second column contains HTML codes

command = "click()"

# Compute class weights after labels are defined
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights))  # Convert to a dictionary for Keras

# Function to preprocess HTML with command
def preprocess_html_with_command(html_codes, command):
    input_ids_batch = []
    attention_mask_batch = []

    for html_code in html_codes:
        # Parse HTML and extract elements
        soup = BeautifulSoup(html_code, 'html.parser')
        elements = []
        for tag in soup.find_all(True):
            tag_name = tag.name
            attrs = tag.attrs
            text = tag.get_text(strip=True)

            elements.append(tag_name)
            for attr, value in attrs.items():
                elements.append(f'{attr}="{value}"')
            if text:
                elements.append(text)

        # Append the command
        elements.append(command)

        # Convert elements to string and tokenize
        processed_html = " ".join(elements)
        tokens = tokenizer.tokenize(processed_html)
        tokens = ['[CLS]'] + tokens + ['[SEP]']

        # Convert tokens to IDs
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)

        # Pad or truncate input_ids and attention_mask
        max_length = 100                #max_length = 500 #max_length = 100 optimized
        if len(input_ids) > max_length:
            input_ids = input_ids[:max_length]
            attention_mask = attention_mask[:max_length]
        else:
            padding_length = max_length - len(input_ids)
            input_ids += [0] * padding_length
            attention_mask += [0] * padding_length

        input_ids_batch.append(input_ids)
        attention_mask_batch.append(attention_mask)

    return np.array(input_ids_batch), np.array(attention_mask_batch)

# Preprocess the batch of HTML codes with the command
input_ids_batch, attention_mask_batch = preprocess_html_with_command(html_codes, command)

# Map unique labels to integers
unique_labels = sorted(set(labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}
label_ids = [label_map[label] for label in labels]

# One-hot encode labels
num_classes = len(label_map)
y_one_hot = tf.keras.utils.to_categorical(label_ids, num_classes=num_classes)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train_ids, X_val_ids, X_train_mask, X_val_mask, y_train, y_val = train_test_split(
    input_ids_batch, attention_mask_batch, y_one_hot, test_size=0.2, random_state=42
)

# Define a custom layer to apply the attention mask
class MaskedEmbedding(Layer):
    def __init__(self, **kwargs):
        super(MaskedEmbedding, self).__init__(**kwargs)

    def call(self, inputs):
        embedding, attention_mask = inputs
        return Multiply()([embedding, tf.cast(tf.expand_dims(attention_mask, -1), dtype=tf.float32)])
#--------------------------------------
# Define the model with regularization
def build_regularized_model(max_length, num_classes):
    input_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

    embedding = Embedding(input_dim=30522, output_dim=128, name='embedding')(input_ids)
    masked_output = MaskedEmbedding(name='masked_embedding')([embedding, attention_mask])

    flatten = Flatten()(masked_output)
    dense1 = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(flatten)
    dropout1 = Dropout(0.5)(dense1)
    dense2 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(dropout1)
    dropout2 = Dropout(0.5)(dense2)
    output = Dense(num_classes, activation='softmax')(dropout2)

    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), #learning_rate = 0.0001
              loss='categorical_crossentropy', metrics=['accuracy'])

    return model

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #eski burası

# Implement early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.000001)

# Build and train the model with regularization
model = build_regularized_model(max_length=100, num_classes=num_classes) #max_length = 500 #max_length = 100 optimized
history = model.fit(
    [X_train_ids, X_train_mask],
    y_train,
    validation_data=([X_val_ids, X_val_mask], y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weights
)

# Evaluate the model on validation data
loss, accuracy = model.evaluate([X_val_ids, X_val_mask], y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Epoch 1/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 12s 204ms/step - accuracy: 0.4640 - loss: 7.2730 - val_accuracy: 0.7417 - val_loss: 5.7432 - learning_rate: 1.0000e-04
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 8s 150ms/step - accuracy: 0.7603 - loss: 5.3452 - val_accuracy: 0.7598 - val_loss: 4.2713 - learning_rate: 1.0000e-04
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 8s 89ms/step - accuracy: 0.8067 - loss: 4.0006 - val_accuracy: 0.7628 - val_loss: 3.2524 - learning_rate: 1.0000e-04
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - accuracy: 0.8599 - loss: 3.0619 - val_accuracy: 0.8649 - val_loss: 2.5611 - learning_rate: 1.0000e-04
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.8897 - loss: 2.4393 - val_accuracy: 0.9189 - val_loss: 2.1028 - learning_rate: 1.0000e-04
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.9412 - loss: 2.0023 - val_accuracy: 0.9129 - val_loss: 1.7778 - learning_rate: 1.0000e-04
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accurac

In [ ]:
#predict etmesi için olan kod

from bs4 import BeautifulSoup

#new_html_code = """<button type="button" class="btn btn-primary" id="action-button" name="submit-btn">Click Me!</button>"""    # id örnek
#new_html_code = """<button type="button" class="close-dialog-window" data-dismiss="modal" aria-label="Close"><span aria-hidden="true">×</span></button>""" #class örnek
#new_html_code = """<button class="back-to-home" id="home-button" aria-label="Go to Home"><span class="home-icon"></span></button>""" #id örnek
new_html_code = """<button class="confirm-btn" name="confirm-data">Confirm Data</button>""" # name örnek

new_input_ids, new_attention_mask = preprocess_html_with_command([new_html_code], command)

prediction = model.predict([new_input_ids, new_attention_mask])
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Map back the predicted class to the label
idx_to_label = {idx: label for label, idx in label_map.items()}
predicted_label = idx_to_label[predicted_class_index]

print(f"Predicted Class Index: {predicted_class_index}")
print(f"Predicted Label: {predicted_label}")

# For debugging: Check distribution of your labels
print("Label Distribution:", Counter(labels))

# For debugging: Print label_map
print("Label Map:", label_map)

#-------------------------------------------------

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(new_html_code, 'html.parser')

# Extract values for id, name, and class attributes

# Function to find the first element with a specific attribute
def find_element_by_attribute(attribute):
    elements_with_attr = soup.find_all(attrs={attribute: True})
    if elements_with_attr:
        return elements_with_attr[0][attribute]
    return None

# Extract the first id, name, and class attribute values from the HTML
predicted_id_value = find_element_by_attribute("id")
predicted_name_value = find_element_by_attribute("name")
predicted_class_value = find_element_by_attribute("class")

#Bulduğumuz değerleri birleştirme
print(f"Predicted Label: {predicted_label}")

if predicted_label=="id":
    print(f'Predicted ID value: {predicted_id_value}')
    print(f'Predicted locator: {predicted_label} = "{predicted_id_value}"') #bulduğu id'i printleme

elif predicted_label=="class":
    print(f'Predicted Class value: {predicted_class_value}')
    print(f'Predicted locator: {predicted_label} = "{predicted_class_value[0]}"') #bulduğu class'ı printleme

elif predicted_label=="name":
    print(f'Predicted Name value: {predicted_name_value}')
    print(f'Predicted locator: {predicted_label} = "{predicted_name_value}"') #bulduğu name'i printleme

else:
    print('No element with an attribute found.')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted Class Index: 2
Predicted Label: name
Label Distribution: Counter({'class': 555, 'id': 554, 'name': 554})
Label Map: {'class': 0, 'id': 1, 'name': 2}
Predicted Label: name
Predicted Name value: confirm-data
Predicted locator: name = "confirm-data"
